1. Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

    - InvoiceNo – номер транзакции
    - StockCode – код товара
    - Description – описание товара
    - Quantity – количество единиц товара, добавленных в заказ
    - InvoiceDate – дата транзакции 
    - UnitPrice – цена за единицу товара
    - CustomerID – id клиента
    - Country – страна, где проживает клиент
https://stepik.org/media/attachments/lesson/361623/data.csv.zip

Импортируйте данные data.csv.zip (ссылка) с кодировкой ISO-8859-1.     
Запишите полученный датафрейм в retail,    
а названия колонок сохраните в переменную retail_columns.    


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
url = "https://stepik.org/media/attachments/lesson/361623/data.csv.zip"
retail = pd.read_csv(url, compression = 'zip',encoding = 'ISO-8859-1' )
retail_columns = retail.columns.tolist()
print(*retail_columns)

InvoiceNo StockCode Description Quantity InvoiceDate UnitPrice CustomerID Country


2. Проверьте, встречаются ли в данных повторяющиеся наблюдения,    
и в качестве ответа укажите их количество.   
Если они есть, то удалите их из retail.   

In [5]:
all_count = retail.shape[0] - retail.drop_duplicates().shape[0]
print( retail.shape[0])
print(all_count)

541909
5268


In [6]:
retail = retail.drop_duplicates()
print(retail.shape[0])

536641


In [7]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


3. Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных.
Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).
Сколько всего заказов отменили пользователи?

In [9]:
retail.InvoiceNo.str.startswith('C').sum()

9251

4.
Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0.
В качестве ответа укажите число оставшихся строк.
531285 541909


In [13]:
tt = retail.query('Quantity > 0')
tt.shape[0]

526054

5. Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

quantile
query
Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.
Подсказки

In [16]:
rg = retail.query('Country == "Germany"').groupby('CustomerID', as_index=False)
rg = rg.agg({'InvoiceNo' : pd.Series.nunique})
gt = rg.InvoiceNo.quantile(0.8)
germany_top = rg.query('InvoiceNo > @gt')['CustomerID']

6. Используя объект с id пользователей (germany_top), полученный на предыдущем шаге,
отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. 
Результирующий датафрейм запишите в top_retail_germany


In [18]:
top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany\
.groupby('StockCode')\
.agg({'Description' : 'count'})\
.sort_values('Description')

,Description
StockCode,
22548,1
22948,1
84800L,1
22956,1
22174,1
...,...
22467,37
22328,45
22423,55


7. Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [26]:
tt = top_retail_germany.groupby('StockCode', as_index=False)
tt.agg({'Description' : 'count'})\
.sort_values('Description')

,StockCode,Description
508,22548,1
711,22948,1
1108,84800L,1
713,22956,1
353,22174,1
...,...,...
474,22467,37
411,22328,45
452,22423,55
409,22326,62


8. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [27]:
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


9. 
Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.
В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)


In [46]:
tt = retail.groupby('InvoiceNo', as_index=False).agg({'Revenue' : 'sum'}).sort_values('Revenue', ascending=False)[:5]
tt.head()

,InvoiceNo,Revenue
22025,581483,168469.60
2303,541431,77183.60
18776,574941,52940.94
19468,576365,50653.91
9741,556444,38970.00


In [48]:
print(*tt.InvoiceNo.to_list())

581483 541431 574941 576365 556444
